| | AIhub | OPUS-100 | TED-2020 | 총합 |
|--|--|--|--|--|
| 데이터 개수* | 6m | 12.5m | 25m | 43.5m |
| json chunk** 범위 | 0~606 | 607~1865 | 1866~4374 | 0~4374 |
| 전처리 전 크기 | 2.5GB | 3.8GB | 8.2GB | 14.5GB |
| 전처리 후 크기 | 4.7GB | 4.4GB | 16.1GB | 25.2GB |

\* m : million, 백만개  
\** chunk는 10,000개 데이터 단위로 수행했음

## 데이터 chunk화

In [1]:
import pandas as pd
import json

with open('final.json', encoding='utf-8') as f:
    data = json.load(f)
    chunkSize = 10000
    for i in range(0, len(data), chunkSize):
        with open('./final_chunk/' + 'final_' + str(i//chunkSize) + '.json', 'w') as outfile:
            json.dump(data[i:i+chunkSize], outfile)

In [ ]:
chunkSize = 10000
for i in range(1866, len(data)+1, chunkSize):
    with open('./final_chunk/' + 'final_' + str(i) + '.json', 'w', encoding='utf-8') as f:
        json.dump(data[i:i+chunkSize], f, ensure_ascii=False)

In [344]:
import pandas as pd
import json
from collections import OrderedDict

# 실험용
# path = 'C:/Users/CPB06GameN/글을쓰자/PyTorch-master/연습폴더/final_chunk/'
# with open(path+'final_300.json', 'r', encoding='utf-8') as f:
#     data = json.load(f, object_pairs_hook=OrderedDict)

## url 전처리 코드

In [197]:
import re
# url 조정
txt = 'https://www.gradient.descent.co.kr dcdcd dcd cdcd dfdfd dcdcd'
txt = 'example.com/etcetc?query=aasd&dest=asds dfdfd'
p = re.compile(r'(((http|https)\:\/\/)?[a-zA-Z0-9\.\-_]+\.([a-zA-Z]){2,6}([a-zA-Z0-9\.\&\/\?\:@\-_=#])*)')
p.sub('얍', txt)

'얍 dfdfd'

## 각 언어별 유니코드

In [12]:
# 동북 아시아
ko = '\u3131-\u3163|\uac00-\ud7af|\u0041-\u007a'    # ko + en
zh = '\u4e00-\u62ff|\u6300-\u77ff|\u7800-\u8cff|\u8d00-\u9fff|\u0041-\u007a' # zh_cn + en
ja = '\u3041-\u3096|\u30a0-\u30ff|\u3400-\u4db5|\u4e00-\u9fcb|\uf900-\ufa6a|\u0041-\u007a'  #ja + en

# 러시아
ru = '\u0410-\u044f'                 # ru

# 유럽 / 미국 (vi, de, pt, fr, id, es, it, en)
for country in 'vi,de,pt,fr,id,es,it,en'.split(','):
    globals()[f'{country}'] = '\u0041-\u007a|\u00c0-\u0178|\u1e00-\u1eff|\u0180-\u024f|\u1e00—\u1eff|\u0027'

# 인도
hi = '\u0900-\u097f|\ua8e0-\ua8ff' # hi (드 파나 가리어)

# 아랍
ar = '\u0627-\u064a'               # ar

## 전처리 함수

In [13]:
import re

def clean_lng(sentence, unicode):
    '''
    문장 전처리 함수
    문장과 해당 언어의 유니코드를 문자열로 넣으면 된다
    '''
    if type(sentence) == str:
        p = re.compile(fr'''
        (
        ((http|https)\:\/\/)?            # http가 존재하거나 존재하지 않거나
        [a-zA-Z0-9\.\-_]+\.              # 웹주소 .kr 이전 까지만 선택
        ([a-zA-Z]){2,6}                  # .kr, .org 등을 모두 선택
        ([a-zA-Z0-9\.\&\/\?\:@\-_=#])*   # 파라미터 선택 / 이메일 선택
        )|
        (
        \([^)]*\)                        # () 안에 내용까지 선택 (반각)    
        )|
        (
        \<[^>]*\>                        # <> 안에 내용까지 선택 (반각)
        )|
        (
        \[[^\]]*\]                       # [] 안에 내용까지 선택 (반각)
        )|
        (
        --[^-]*--                        # -- 내용 -- 내용까지 선택 (반각)
        )|
        (
        \u00ab[^\u00bb]*\u00bb           # <> 안에 내용까지 선택 (반각)
        )|
        (
        \uff1c[^\uff1e]*\uff1e           # < > 안에 내용까지 선택 (전각)
        )|
        (
        \ufe64[^\ufe65]*\ufe65           # << >> 안에 내용까지 선택 (전각)
        )|
        (
        \uff08[^\uff09]*\uff09           # () 안에 내용까지 선택 (전각)    
        )|
        (
        \ufe59[^\ufe5a]*\ufe5a            # () 안에 내용까지 선택 (전각) 
        )|
        (
        \uff3b[^\uff3d]*\uff3d           # [] 안에 내용까지 선택 (전각)
        )|
        (                                
        [^ {unicode}|  # 사용할 언어

        0-9|0-9 /|0-9/|0-9 :|0-9:|       # 숫자와 숫자 뒤 /와 :는 선택에서 제외

        ·|.|,|!|?|"|"|'|⸢|⸥|。|          # 포함할 특수문자 (반각)
                                         # 포함할 특수문자 (전각) :
        \uff0e|\ufe52|\uff0c|\ufe51|     # 마침표, 콤마, 느낌표,
        \ufe50|\ufe57|\uff01|\ufe15|     # 물음표, 큰따옴표,
        \uff1f|\ufe56|\ufe16|\uff02|     # 어퍼스트로피, 꺽새 (전각)
        \uff07|\ufe41|\ufe42|\ufe43|\ufe44]
        )                               
        ''', re.VERBOSE)
        result = p.sub('', sentence)
    else:
        result = ''
    return result

In [14]:
def cleansing_data(chunk_data):
    '''
    chunk json 넣어주시면 전처리해서 새로운 OrderedDict를 반환합니다
    '''
    new = []
    for line in chunk_data:
#         if list(filter(None, '') ) == []:
#             continue
#         if re.search(r'[0-9]', line) is None:
        new_line = OrderedDict()
        src_unicode = globals()[line['src_lang']]
        tgt_unicode = globals()[line['tgt_lang']]

        new_line['src_lang'] = line['src_lang']
        new_line['src_text_raw'] = line['src_text_raw']
        new_line['src_text'] = clean_lng(line['src_text_raw'], src_unicode)

        new_line['tgt_lang'] = line['tgt_lang']
        new_line['tgt_text_raw'] = line['tgt_text_raw']
        new_line['tgt_text'] = clean_lng(line['tgt_text_raw'], tgt_unicode)

        new_line['origin'] = line['origin']
        new_line['domain'] = line['domain']
        new.append(new_line)
    return new

In [15]:
import os
import json
from collections import OrderedDict

path = 'C:/Users/CPB06GameN/글을쓰자/PyTorch-master/연습폴더/final_chunk'
files = os.listdir(path)

save_path = 'C:/Users/CPB06GameN/글을쓰자/PyTorch-master/연습폴더/cleansing_json/'

for idx in range(len(files)):
    with open(path + f'/final_{idx}.json', 'r', encoding='utf-8') as f:
        data = json.load(f, object_pairs_hook=OrderedDict)
    cleansing_json = cleansing_data(data)
    with open(save_path + f'final_{idx}.json', 'w', encoding='utf-8') as f2:
        json.dump(cleansing_json, f2, ensure_ascii=False, indent=4)

In [3]:
import json


save_path = 'C:/Users/CPB06GameN/글을쓰자/PyTorch-master/연습폴더/cleansing_json/'

with open(save_path+'final_52.json', 'r', encoding='utf-8') as f:
    tmp = json.load(f)

In [10]:
temp = []
temp.extend([2,3,4])
temp

[2, 3, 4]

## 전처리 연습

In [313]:
p = re.compile(r'[^ \u3131-\u3163|\uac00-\ud7af]') # ko
p = re.compile(r'[^ \u0041-\u007a]') # en
p = re.compile(r'[^ \u4e00-\u62ff|\u6300-\u77ff|\u7800-\u8cff|\u8d00-\u9fff]') # zh_cn

p = re.compile(r'[^ \u0410-\u044f]') # ru
p = re.compile(r'[^ \u3041-\u3096|\u30a0-\u30ff|\u3400-\u4db5|\u4e00-\u9fcb|\uf900-\ufa6a]') #ja
p = re.compile(r'[^ \u0041-\u007a|\u00c0-\u0178|\u1e00-\u1eff|\u0180-\u024f|\u1e00—\u1eff|\u0027]') # vi, de, pt, fr, id, es, it, en
p = re.compile(r'[^ \u0900-\u097f|\ua8e0-\ua8ff]') # hi (드 파나 가리어)
p = re.compile(r'[^ \u0627-\u064a]') # ar


test_ko1 = 'Pepsi 수익성장의 2 / 3 는 2/3 개선 식품에서 발생,, 합니다 .﹙ 아뵤﹚'
test_ko2 = '( 음악 ) ♫ 내가 아침에 ♫ ♫ 일어났을 때 ♫ ♫ 커피를 따르고 ♫ ♫ 신문을 읽고 ♫ ♫ 그리곤 지막인 날이 될 것처럼 ♫ ( 박수 ) 자 그럼 .'

test_en1 = '''♫  I 've later come to know that to be the collective socialization of men , better known as the " man box . "'''
test_en2 = '( Audience : Man . ㅂ ♫ )'

test_zh1 = '♫ （ 笑声 ） （ 掌声 ） 我给你讲个小故事来形容我现在是什么样的'
test_zh2 = '♫ 那个男的接着说  ， “ 他已经非常努力了 ， 不是吗 ？ “ 的妻子Tipper 。'

test_es1 = '♫  Estos son mis dos hijos : Kendall y Jay .'
test_es2 = ':♫  " ¿ Por qué lloras ?'

test_fr1 = '♫  -- Rires -- Mettez-vous à ma place ! œ'
test_fr2 = '♫  Chris Anderson : Marché conclu !'

test_ru1 = '♫ Эта теории заключает в себе все элементы , обладание которыми , как мы часто полагаем , и означает быть " настоящим мужчиной " .'
test_ru2 = '♫  ( аудитория : Мужчина . ) " как мужчина . "'

test_id1 = '♫ ( Tertawa ) Cobalah kalian mengerti perasaan saya !'
test_id2 = '♫  Dan dia bilang " Benar , itu adalah mantan wakil presiden Al Gore dan istrinya Tipper . "'

test_ja1 = '♫  脳はご存じの通りニューロン （ 神経細胞 ） から 構成されています'
test_ja2 = '♫ 「 すいません この道は何という名前でしょう ？ 」'

test_vi1 = '♫ ( Cười ) Câu chuyện này là có thật -- đến từng chi tiết .'
test_vi2 = '♫ Gọi cho Washington ! "'

test_de1 = '♫ Ich war bereit , ein Diplomat , Lehrer , Arzt zu werden -- alles war bereit .'
test_de2 = '♫ ( Lachen ) " Was ist los ?'

test_it1 = '♫ Ne ho bisogno ! !'
test_it2 = '''♫  E l' uomo disse : " E ’ caduto parecchio in basso , vero ? "'''

test_hi1 = '♫ ( हँस ) स र दुन य के अवसर मेरे स मने थे ।'
test_hi2 = '" ♫ प ँच स ल तक कुँए ख देग ?'

test_pt1 = '♫  — sem motorista ...'
test_pt2 = '♫ ( Risos ) Estávamos num Ford Taurus alugado .'

test_ar1 = '♫ ( فيديو ) سلمان خان : إذاً وتر المثلث سيكون 5'
test_ar2 = '♫ ( ضحك ) ولنتوقف هنا لوهلة هذا الشخص قام بعملية التفاضل [△|㏊|ℓ|▶|µ|★|\(|\)|\[|\]|▽|♦|•]'




lng = '\u3131-\u3163|\uac00-\ud7af'
p = re.compile(fr'''
(
((http|https)\:\/\/)?            # http가 존재하거나 존재하지 않거나
[a-zA-Z0-9\.\-_]+\.              # 웹주소 .kr 이전 까지만 선택
([a-zA-Z]){2,6}                  # .kr, .org 등을 모두 선택
([a-zA-Z0-9\.\&\/\?\:@\-_=#])*   # 파라미터 선택 / 이메일 선택
)|
(
\([^)]*\)                        # () 안에 내용까지 선택 (반각)    
)|
(
\<[^>]*\>                        # <> 안에 내용까지 선택 (반각)
)|
(
\[[^\]]*\]                       # [] 안에 내용까지 선택 (반각)
)|
(
--[^-]*--                        # -- 내용 -- 내용까지 선택 (반각)
)|
(
\u00ab[^\u00bb]*\u00bb           # <> 안에 내용까지 선택 (반각)
)|
(
\uff1c[^\uff1e]*\uff1e           # < > 안에 내용까지 선택 (전각)
)|
(
\ufe64[^\ufe65]*\ufe65           # << >> 안에 내용까지 선택 (전각)
)|
(
\uff08[^\uff09]*\uff09           # () 안에 내용까지 선택 (전각)    
)|
(
\ufe59[^\ufe5a]*\ufe5a            # () 안에 내용까지 선택 (전각) 
)|
(
\uff3b[^\uff3d]*\uff3d           # [] 안에 내용까지 선택 (전각)
)|
(                                
[^ {lng}|  # 사용할 언어

0-9|0-9 /|0-9/|0-9 :|0-9:|       # 숫자와 숫자 뒤 /와 :는 선택에서 제외

·|.|,|!|?|"|"|'|⸢|⸥|             # 포함할 특수문자 (반각)
                                 # 포함할 특수문자 (전각) :
\uff0e|\ufe52|\uff0c|\ufe51|     # 마침표, 콤마, 느낌표,
\ufe50|\ufe57|\uff01|\ufe15|     # 물음표, 큰따옴표,
\uff1f|\ufe56|\ufe16|\uff02|     # 어퍼스트로피, 꺽새 (전각)
\uff07|\ufe41|\ufe42|\ufe43|\ufe44]
)                               
''', re.VERBOSE)

p.sub('', test_ko1)

' 수익성장의 2 / 3 는 2/3 개선 식품에서 발생,, 합니다 .'